In [11]:
import pandas as pd
import re

In [2]:
# read files: business, review, tip, user
business = pd.read_json("../data/business_city.json", lines = True)

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
1,7Dv4_HAxsxvadEsT5fxQBg,Dependable Brakes & Exhaust,1110 Saw Mill Run Blvd,Pittsburgh,PA,15226,40.406667,-80.004450,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Automotive, Auto Repair","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
2,M_guz7Dj7hX0evS672wIwA,Chocolate Shoppe Ice Cream,2831 Parmenter St,Middleton,WI,53562,43.105310,-89.510142,3.5,6,1,"{'BikeParking': 'True', 'Caters': 'True', 'Whe...","Desserts, Food, Ice Cream & Frozen Yogurt","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
3,03x6ZlJ7s39DHqfTJU1Ecg,Beechmont Country Club,29600 Chagrin Blvd,Beachwood,OH,44122,41.461448,-81.470127,4.5,7,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Event Planning & Services, Venues & Event Spac...",None
4,_4Oe9V-qTpU5iemM9bphlA,Goodyear Auto Service Center,"800 Fort Duquesne Blvd, Ste 1",Pittsburgh,PA,15222,40.444614,-79.999344,3.5,12,1,{'BusinessAcceptsCreditCards': 'True'},"Oil Change Stations, Auto Repair, Automotive, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."


In [110]:
steak_business = business[business["categories"].str.lower().str.contains("steakhouse") == True]
steak_business = steak_business[["name", "stars", "attributes"]]
steak_business.head()

,name,stars,attributes
340,Nobu Tei,3.5,"{'Alcohol': ''none'', 'RestaurantsDelivery': '..."
342,Ferris Steak House,3.5,"{'RestaurantsReservations': 'True', 'Restauran..."
741,Grandview Saloon,3.0,"{'Alcohol': 'u'full_bar'', 'WiFi': ''free'', '..."
931,DeBlaze Grill,4.5,"{'RestaurantsAttire': 'u'casual'', 'Ambience':..."
1305,Mariner's Inn,3.5,"{'WiFi': 'u'free'', 'RestaurantsGoodForGroups'..."


In [111]:
# one record contain NA in attributes, remove this record
steak_business = steak_business.dropna()
# reset index
steak_business = steak_business.reset_index()[["name", "stars", "attributes"]]

In [113]:
# extract attributes
attributes = []
for ind in steak_business.index: 
    attr = steak_business.loc[ind, "attributes"].keys()
    attributes.extend(list(attr))

attributes = list(set(attributes))

In [114]:
attr_dict = {}
for attr in attributes: 
    attr_dict[attr] = []
for ind in steak_business.index:
    attribute = steak_business.loc[ind, "attributes"]
    for attr in attributes: 
        if attr in attribute.keys(): 
            attr_dict[attr].append(attribute[attr])
        else: 
            attr_dict[attr].append("NaN")

In [131]:
attr_df = pd.DataFrame(attr_dict)
steak_attributes = pd.concat([steak_business, attr_df], axis = 1)
steak_attributes.head()

,name,stars,attributes,NoiseLevel,Ambience,RestaurantsPriceRange2,DietaryRestrictions,RestaurantsReservations,OutdoorSeating,RestaurantsAttire,...,RestaurantsTakeOut,BestNights,GoodForMeal,GoodForKids,HasTV,RestaurantsGoodForGroups,CoatCheck,BYOB,Smoking,HappyHour
0,Nobu Tei,3.5,"{'Alcohol': ''none'', 'RestaurantsDelivery': '...",'quiet',"{'romantic': False, 'intimate': False, 'classy...",2,NaN,True,False,'casual',...,True,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",True,True,True,NaN,NaN,NaN,NaN
1,Ferris Steak House,3.5,"{'RestaurantsReservations': 'True', 'Restauran...",NaN,NaN,3,NaN,True,True,u'casual',...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Grandview Saloon,3.0,"{'Alcohol': 'u'full_bar'', 'WiFi': ''free'', '...",u'average',"{'romantic': False, 'intimate': False, 'classy...",2,NaN,True,True,u'casual',...,True,"{'monday': False, 'tuesday': False, 'friday': ...","{'dessert': False, 'latenight': False, 'lunch'...",True,True,True,NaN,NaN,NaN,True
3,DeBlaze Grill,4.5,"{'RestaurantsAttire': 'u'casual'', 'Ambience':...",NaN,"{'touristy': False, 'hipster': False, 'romanti...",2,NaN,NaN,False,u'casual',...,True,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",True,True,True,NaN,NaN,NaN,True
4,Mariner's Inn,3.5,"{'WiFi': 'u'free'', 'RestaurantsGoodForGroups'...",u'quiet',"{'touristy': False, 'hipster': False, 'romanti...",3,NaN,True,True,u'casual',...,True,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",False,True,True,NaN,False,NaN,True


In [136]:
na_num = {}
for col in attr_df.columns: 
    na_num[col] = sum(attr_df[col] == "NaN")
na_num

{'NoiseLevel': 43,
 'Ambience': 22,
 'RestaurantsPriceRange2': 13,
 'DietaryRestrictions': 352,
 'RestaurantsReservations': 9,
 'OutdoorSeating': 16,
 'RestaurantsAttire': 19,
 'GoodForDancing': 304,
 'BusinessAcceptsCreditCards': 7,
 'DogsAllowed': 291,
 'Corkage': 343,
 'RestaurantsDelivery': 12,
 'DriveThru': 314,
 'WiFi': 50,
 'Alcohol': 23,
 'WheelchairAccessible': 285,
 'AgesAllowed': 352,
 'BusinessAcceptsBitcoin': 310,
 'Music': 268,
 'ByAppointmentOnly': 295,
 'RestaurantsTableService': 189,
 'BikeParking': 58,
 'BYOBCorkage': 339,
 'Caters': 89,
 'BusinessParking': 12,
 'RestaurantsTakeOut': 19,
 'BestNights': 298,
 'GoodForMeal': 79,
 'GoodForKids': 17,
 'HasTV': 19,
 'RestaurantsGoodForGroups': 14,
 'CoatCheck': 286,
 'BYOB': 346,
 'Smoking': 316,
 'HappyHour': 166}

In [137]:
# keep Ambience, RestaurantsPriceRange2, RestaurantsReservations, OutdoorSeating, RestaurantsAttire, BusinessAcceptsCreditCards, 
# RestaurantsDelivery, Alcohol, BusinessParking, RestaurantsTakeOut, GoodForKids, GoodForKids, HasTV, RestaurantsGoodForGroups
steak_attributes = steak_attributes[["name", "stars", "Ambience", "RestaurantsPriceRange2", "RestaurantsReservations", 
                                     "OutdoorSeating", "RestaurantsAttire", "BusinessAcceptsCreditCards", "RestaurantsDelivery", 
                                     "Alcohol", "BusinessParking", "RestaurantsTakeOut", "GoodForKids", "GoodForKids", "HasTV", 
                                     "RestaurantsGoodForGroups"]]
steak_attributes.head()

,name,stars,Ambience,RestaurantsPriceRange2,RestaurantsReservations,OutdoorSeating,RestaurantsAttire,BusinessAcceptsCreditCards,RestaurantsDelivery,Alcohol,BusinessParking,RestaurantsTakeOut,GoodForKids,GoodForKids,HasTV,RestaurantsGoodForGroups
0,Nobu Tei,3.5,"{'romantic': False, 'intimate': False, 'classy...",2,True,False,'casual',True,False,'none',"{'garage': False, 'street': False, 'validated'...",True,True,True,True,True
1,Ferris Steak House,3.5,NaN,3,True,True,u'casual',True,NaN,u'full_bar',"{'garage': False, 'street': False, 'validated'...",True,NaN,NaN,NaN,NaN
2,Grandview Saloon,3.0,"{'romantic': False, 'intimate': False, 'classy...",2,True,True,u'casual',True,False,u'full_bar',"{'garage': False, 'street': True, 'validated':...",True,True,True,True,True
3,DeBlaze Grill,4.5,"{'touristy': False, 'hipster': False, 'romanti...",2,NaN,False,u'casual',True,True,u'full_bar',"{'garage': False, 'street': True, 'validated':...",True,True,True,True,True
4,Mariner's Inn,3.5,"{'touristy': False, 'hipster': False, 'romanti...",3,True,True,u'casual',True,False,'full_bar',"{'garage': False, 'street': False, 'validated'...",True,False,False,True,True


In [142]:
steak_attributes.to_csv("../data/steak_attributes.csv")